# Title
#### This notebook analyzes credit card customer data from Edulyt India. The goal is to extract insights on spending, repayment, and bank profitability.

# Data Loading

#### We will upload our .xls file and convert it into .csv file.
#### Now we will check the structure of converted .csv file.

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install xlrd

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
xls_file = 'Credit Banking_Project - 1.xls'
df = pd.read_excel(xls_file, sheet_name = 0, engine='xlrd')
df.to_csv('converted_file.csv', index= False)

In [ ]:
print("Conversion done!")

In [ ]:
df= pd.read_csv("converted_file.csv")

In [ ]:
print(df.info())

In [ ]:
print(df.head())

In [ ]:
xls_file2 = 'Spending.xls'
spend_df = pd.read_excel(xls_file2, sheet_name = 0, engine='xlrd')
spend_df.to_csv('spended_file.csv', index= False)

In [ ]:
print("Conversion done!")

In [ ]:
print(spend_df.info())

In [ ]:
print(spend_df.head())

In [ ]:
xls_file3 = 'Repayment.xls'
txn_df = pd.read_excel(xls_file3, sheet_name = 0, engine='xlrd')
txn_df.to_csv('txn_file.csv', index= False)

In [ ]:
print("Conversion done!")
print(txn_df.info())

In [ ]:
print(txn_df.head())

# Data Cleaning

#### Here we will detect the age group and fix them as the Age<18.

In [ ]:
df= df[df['Age'] >=18]

In [ ]:
df['Age_Group']= pd.cut(df['Age'],
                        bins=[18,25,35,45,60,100],
                        labels=['18-25', '26-35', '36-45', '46-60', '60+'])

In [ ]:
print(df['Age_Group'].head())

# Load/ Join Monthly spend Data

#### Here we will upload the Spending and Repayment files.
#### And merge them on the basis of Costomer

In [ ]:
spend_df= pd.read_csv("spended_file.csv")

In [ ]:
print(spend_df.head())

In [ ]:
print("df columns:", df.columns.tolist())
print("spend_df columns:", spend_df.columns.tolist())

In [ ]:
df_combined = pd.merge(txn_df, spend_df, on="Costomer", how="left")

In [ ]:
df.columns = df.columns.str.strip()
spend_df.columns = spend_df.columns.str.strip()

In [ ]:
print(df_combined.info())

In [ ]:
df_combined.columns = df_combined.columns.str.strip()  # remove extra spaces

In [ ]:
print(df_combined.head())

# Monthly Calculations

#### Now check every transaction on the basis of Monthly Spend, Monthly Repayment, Over limit Check and Interest Calculation(2.9%)

In [ ]:
print(df_combined.columns)

In [ ]:
df_combined.rename(columns={
    'Amount_x': 'Spend',
    'Amount_y': 'Repayment',
    'Month_x': 'Month'
}, inplace=True)

In [ ]:
print(df_combined.columns)

In [ ]:
monthly_spend= df_combined.groupby(['Costomer', 'Month',])['Spend'].sum().reset_index()

In [ ]:
print(monthly_spend)

In [ ]:
print(monthly_spend.info())

In [ ]:
print(monthly_spend.head())

In [ ]:
monthly_comparison = df_combined.groupby('Month')[['Spend', 'Repayment']].sum().reset_index()
plt.figure(figsize=(12,6))
sns.lineplot(data=pd.melt(monthly_comparison, id_vars='Month'), x='Month', y='value', hue='variable')
plt.title("Monthly Spend vs. Repayment")
plt.ylabel("Amount (₹)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Insights & Visualizations

#### Here check all the Top 10 spenders on the basis of Segment-wise spend, Age-group spend, Category spend, Most profitable segment

In [ ]:
top10 = df_combined.groupby('Costomer')['Spend'].sum().sort_values(ascending=False).head(10) 

In [ ]:
print(top10)

In [ ]:
sns.set(style='whitegrid')

In [ ]:
top10 = df_combined.groupby('Costomer')['Spend'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10,6))
sns.barplot(x=top10.index, y=top10.values, palette='viridis')
plt.title("Top 10 Paying Costomers")
plt.xlabel("Costomer ID")
plt.ylabel("Total Spend")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
print(df.columns)

# Bank Profit Calculation

#### If (Repayment < Spend) then we put interest on it and we will calculate the profit of the bank in every month. 

In [ ]:
df_combined['Due'] = df_combined['Spend'] - df_combined['Repayment']
df_combined['Due'] = df_combined['Due'].apply(lambda x: x if x > 0 else 0)
df_combined['Interest'] = df_combined['Due'] * 0.029

In [ ]:
print(df_combined)

In [ ]:
monthly_profit = df_combined.groupby('Month')['Interest'].sum()
plt.figure(figsize=(12,6))
monthly_profit.plot(kind='line', marker='o', color='green')
plt.title("Monthly Bank Profit from Interest (2.9% Rate)")
plt.xlabel("Month")
plt.ylabel("Profit (₹)")
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()